# Data Cleaning for Anime and Revenue 

Prepared by: Danny Hinojosa
Dated: June 29, 2022

## Introduction

This notebook is a part of a series of Juptyer notebooks to create a predictive machine learning model. The purpose of this notebook is to load raw data regarding Anime shows and movies, and transform the data for Exploratory Data Analysis (EDA). The data was collected from various sources from Kaggle datasets, somanithing.com, and other websites through web scrapping. A purpose of the transformation is to condense the information. The information will also be kept based on what features will be available before anime is broadcast or premiered at the movies. Second, a process will be created to parse various datasets and combine them into one csv file.

The goal of the overall project is to create a predictive model for revenue and determine what features are important for the model's decision making. To that end, the data clean up will involve keeping as much data as possible regarding revenue. 

## Review of the dataset source 

The following dataset were used:

- Kaggle Anime Dataset from <u>Animelist</u>: `data/AnimeList.csv`

- Kaggle Anime Dataset from <u>Animelist</u>: `data/anime.csv'`

- Kaggle Anime Dataset from <u>Animelist</u>: `data/anime.csv.zst/anime.csv.zst`

    - Required additional function `read_zstd` provided by Kaggle 

- Anime Revenue for TV shows from <u>someanithing.com</u>: `Data/someanithing.com_rev.xlsx`

- Anime Revenue for films from <u>someanithing.com</u>: `Data/someanithing.com_film.xlsx`

- Webs crapping: Web scraping processed in supplement notebook
    - Box office for anime movies from <u>eiren.com</u> :`importdata/eiren.csv`
    - Box office for anime movies from <u>numbers.com</u> :`importdata/numbers.csv`
    - Box office for anime movies from <u>wikipedia.com</u> :`importdata/wikipedia.csv`
    

Note: All the datasets from Animelist will be combined. All other datasets will need a new key to pair the data from Animelist.

### Import packages and library for python

In [1]:
#import important packages

#import pandas and numpy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#import other packages
import zstandard as zstd
from collections import Counter
import re

#Import function from kaggle
def read_zstd(path: str):
    with open(path, "rb") as f:
        return pd.read_csv(zstd.ZstdDecompressor().stream_reader(f))


### Import all dataset

In [2]:
# import all datasets 

#dataset 1
anime1_df = pd.read_csv('data/AnimeList.csv')

#dataset 2 - require tab sepertor

anime2_df = pd.read_csv('data/anime.csv', on_bad_lines='skip', sep ='\t')

#dataset 3 - require zstd to open
anime3_df = read_zstd("data/anime.csv.zst/anime.csv.zst")

#dataset 4 - tv show revenue
anime_1rev_df = pd.read_excel('Data/someanithing.com_rev.xlsx')

#dataset 5 - film revenue
anime_1film_df = pd.read_excel('Data/someanithing.com_film.xlsx')

#dataset 6 - film revenue
anime_2film_df = pd.read_csv('importdata/eiren.csv', header=1)

#dataset 7 - film revenue
anime_3film_df = pd.read_csv('importdata/numbers.csv')

#dataset 8 - film revenue
anime_4film_df = pd.read_csv('importdata/wikipedia.csv', header=0)

## Review dataset from animelist

### Review 1st dataset from Animelist

The first dataset from the Animelist will be parsed to look for common keys. 

Data removal will consider the following:
- Data that is unique to the show will be removed with the exception of the show title. Unique data is considered  as traits or information that is not repeated among other entries of data. 
- Data that would have been unknown at the time of the show's premiers will also be removed. 
- Repetitive data relating to the release or end time of the show will be removed.

Removal of data will be primarily perform on columns of data (*features*).

### Check header and data

In [3]:
#Create copy... use _rev for copy

anime1_df_rev= anime1_df.copy()
#Display first 3 lines
display(anime1_df_rev.iloc[:,0:15].head(2))
display(anime1_df_rev.iloc[:,15:].head(2))
#list columns using info method
anime1_df_rev.info(10)

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,airing,aired_string,aired,duration,rating
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,False,"Jan 13, 2012 to Mar 30, 2012","{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,False,"Apr 2, 2007 to Oct 1, 2007","{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older


,score,scored_by,rank,popularity,members,favorites,background,premiered,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme
0,7.63,139250,1274.0,231,283882,2809,Inu x Boku SS was licensed by Sentai Filmworks...,Winter 2012,Fridays at Unknown,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen","['""Nirvana"" by MUCC']","['#1: ""Nirvana"" by MUCC (eps 1, 11-12)', '#2: ..."
1,7.89,91206,727.0,366,204003,2579,NaN,Spring 2007,Unknown,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen","['""Romantic summer"" by SUN&LUNAR']","['#1: ""Ashita e no Hikari (明日への光)"" by Asuka Hi..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14478 entries, 0 to 14477
Data columns (total 31 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anime_id        14478 non-null  int64  
 1   title           14478 non-null  object 
 2   title_english   5724 non-null   object 
 3   title_japanese  14443 non-null  object 
 4   title_synonyms  8937 non-null   object 
 5   image_url       14382 non-null  object 
 6   type            14478 non-null  object 
 7   source          14478 non-null  object 
 8   episodes        14478 non-null  int64  
 9   status          14478 non-null  object 
 10  airing          14478 non-null  bool   
 11  aired_string    14478 non-null  object 
 12  aired           14478 non-null  object 
 13  duration        14478 non-null  object 
 14  rating          14478 non-null  object 
 15  score           14478 non-null  float64
 16  scored_by       14478 non-null  int64  
 17  rank            12904 non-null 

#### Review data and remove unnecessary data

The data was reviewed and judged based on the title of the columns. The following are takeaways: 

- Score and popularity related metrics would have been unknown prior to the premier. These columns will be removed

- Theme songs and images are unique to show and will be removed.

- Broadcast, premier times, and Aired string are repeated information about release time. This information is already present in the aired column. These columns will be removed


In [4]:
#remove confusing data


columns_drop = ['title_japanese',                                         #'anime_id', 'title', 'title_english', 'title_japanese',
                'image_url',                                             #'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
                'aired_string','score',                                                                 #'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',      
                'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',  #'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
                'premiered', 'broadcast',                                                 #'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio', 
                'opening_theme', 'ending_theme']                                          #'genre', 'opening_theme', 'ending_theme'

anime1_df_rev.drop(columns = columns_drop, inplace = True)

#### Check data types

Reexamine the data set using header and information to look at data type.

In [5]:
anime1_df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14478 entries, 0 to 14477
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   anime_id        14478 non-null  int64 
 1   title           14478 non-null  object
 2   title_english   5724 non-null   object
 3   title_synonyms  8937 non-null   object
 4   type            14478 non-null  object
 5   source          14478 non-null  object
 6   episodes        14478 non-null  int64 
 7   status          14478 non-null  object
 8   airing          14478 non-null  bool  
 9   aired           14478 non-null  object
 10  duration        14478 non-null  object
 11  rating          14478 non-null  object
 12  related         14478 non-null  object
 13  producer        8288 non-null   object
 14  licensor        3373 non-null   object
 15  studio          8544 non-null   object
 16  genre           14414 non-null  object
dtypes: bool(1), int64(2), object(14)
memory usage: 1.8

In [6]:
anime1_df_rev.head(2)

,anime_id,title,title_english,title_synonyms,type,source,episodes,status,airing,aired,duration,rating,related,producer,licensor,studio,genre
0,11013,Inu x Boku SS,Inu X Boku Secret Service,Youko x Boku SS,TV,Manga,12,Finished Airing,False,"{'from': '2012-01-13', 'to': '2012-03-30'}",24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 17207, 'type': 'man...","Aniplex, Square Enix, Mainichi Broadcasting Sy...",Sentai Filmworks,David Production,"Comedy, Supernatural, Romance, Shounen"
1,2104,Seto no Hanayome,My Bride is a Mermaid,The Inland Sea Bride,TV,Manga,26,Finished Airing,False,"{'from': '2007-04-02', 'to': '2007-10-01'}",24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 759, 'type': 'manga...","TV Tokyo, AIC, Square Enix, Sotsu",Funimation,Gonzo,"Comedy, Parody, Romance, School, Shounen"


It is expected aired time will be split into two features for start and finish. As this will increase the number of features (columns), it would be preferred to split at this step.

Furthermore the data type of the new features will be revised to datetime .

In [7]:
#split airing to just pair of text and clean the str
anime1_df_rev[['aired_start','aired_end']]= anime1_df_rev['aired'].str.split(',', expand = True)
anime1_df_rev['aired_start']= anime1_df_rev['aired_start'].str.replace("{'from': ", "",regex=True)
anime1_df_rev['aired_end']= anime1_df_rev['aired_end'].str.replace("'to': ", "",regex=True)
anime1_df_rev['aired_start']= anime1_df_rev['aired_start'].str.replace("['{}']", "",regex=True)
anime1_df_rev['aired_end']= anime1_df_rev['aired_end'].str.replace("['{}']", "",regex=True)

#data type correct for datatime
anime1_df_rev['aired_start'] = pd.to_datetime(anime1_df_rev['aired_start'], errors='coerce')
anime1_df_rev['aired_end'] = pd.to_datetime(anime1_df_rev['aired_end'], errors='coerce')

Drop old columns and check new column data type

In [8]:
#drop as airing and aired 
anime1_df_rev.drop(columns=['aired','airing'], inplace=True)
anime1_df_rev[['aired_start','aired_end']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14478 entries, 0 to 14477
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   aired_start  12787 non-null  datetime64[ns]
 1   aired_end    12287 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 226.3 KB


More exploration will be required at the EDA, especially for the `related` feature. 

### Check ID for uniqueness

Anime ID was provided in the dataset. This feature will be checked to confirm if the Anime ID's are unique.

In [9]:
#check index
print(f"Anime ID is unique :{len(anime1_df_rev['anime_id'].unique())==anime1_df_rev.shape[0]}")

Anime ID is unique :True


Anime ID will be used as index within Anime list data.

In [10]:
# Replace the index with anime index
anime1_df_rev.set_index('anime_id',inplace = True)

### Review 2nd dataset from Animelist

The second dataset from Animelist will be parsed to look for common keys. Data removal will consider the following:

- Data that is unique to the show will be removed with the exception of the show title. Unique data is considered as traits or information that is not repeated among other entries of data.
- Data that would have been unknown at the time of the show's premiers will also be removed.
- Repetitive data relating to the release or end time of the show will be removed.

### Check header and data

In [11]:
#Create copy... use _rev for copy
anime2_df_rev = anime2_df.copy()

#Review top 3 lines, check shape and columns
display(anime2_df_rev.iloc[:,0:15].head(3))
display(anime2_df_rev.iloc[:,15:].head(3))
display(anime2_df_rev.shape)
display(anime2_df_rev.info(10))

,anime_id,anime_url,title,synopsis,main_pic,type,source_type,num_episodes,status,start_date,end_date,season,studios,genres,score
0,2366,https://myanimelist.net/anime/2366/Touma_Kishi...,Touma Kishinden Oni,Shuramaru is hated and feared by the villagers...,https://cdn.myanimelist.net/images/anime/9/829...,TV,Game,25.0,Finished Airing,1995-10-05 00:00:00,1996-03-21 00:00:00,Fall 1995,J.C.Staff,Supernatural,NaN
1,4940,https://myanimelist.net/anime/4940/Sabaku_no_K...,Sabaku no Kaizoku! Captain Kuppa,"Sometime in the future, the world was complete...",https://cdn.myanimelist.net/images/anime/9/736...,TV,Manga,26.0,Finished Airing,2001-08-13 00:00:00,2002-02-11 00:00:00,Summer 2001,NaN,Action|Adventure,NaN
2,50285,https://myanimelist.net/anime/50285/On_Air_Dek...,On Air Dekinai!,"Set in 2014, the anime follows the adventures ...",https://cdn.myanimelist.net/images/anime/1021/...,TV,Manga,12.0,Currently Airing,2022-01-10 00:00:00,2022-03-28 00:00:00,Winter 2022,Space Neko Company|Jinnan Studio,Comedy,NaN


,score_count,score_rank,popularity_rank,members_count,favorites_count,watching_count,completed_count,on_hold_count,dropped_count,plan_to_watch_count,...,score_08_count,score_07_count,score_06_count,score_05_count,score_04_count,score_03_count,score_02_count,score_01_count,clubs,pics
0,NaN,NaN,11743,884,0,61,173,31,75,544,...,0,0,0,0,0,0,0,0,14045|10778|342,https://cdn.myanimelist.net/images/anime/9/544...
1,NaN,NaN,13321,491,2,26,124,18,73,250,...,0,0,0,0,0,0,0,0,8494,https://cdn.myanimelist.net/images/anime/9/736...
2,NaN,NaN,9603,2057,8,203,0,39,88,1725,...,0,0,0,0,0,0,0,0,27907|8652,https://cdn.myanimelist.net/images/anime/1021/...


(13379, 38)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13379 entries, 0 to 13378
Data columns (total 38 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   anime_id             13379 non-null  int64  
 1   anime_url            13379 non-null  object 
 2   title                13379 non-null  object 
 3   synopsis             13375 non-null  object 
 4   main_pic             13379 non-null  object 
 5   type                 13379 non-null  object 
 6   source_type          13379 non-null  object 
 7   num_episodes         12948 non-null  float64
 8   status               13379 non-null  object 
 9   start_date           13198 non-null  object 
 10  end_date             12799 non-null  object 
 11  season               4280 non-null   object 
 12  studios              10343 non-null  object 
 13  genres               13379 non-null  object 
 14  score                10714 non-null  float64
 15  score_count          10714 non-null 

None

#### Review data and remove unnecessary data

The data was reviewed and judged based on the title of the columns. The following are takeaways:

- Anime url, and picture are unique to the shows and are not required.
- Scores and related metric will be removed .
- Status, and Season will be removed as this is repeated information per start and end date. 

In [12]:
columns_drop = [ 'anime_url',   'main_pic',        
                
                'season','score', 'score_count', 'score_rank',
                'popularity_rank', 'members_count', 'favorites_count', 'watching_count',
                'completed_count', 'on_hold_count', 'dropped_count',
                'plan_to_watch_count', 'total_count', 'score_10_count',
                'score_09_count', 'score_08_count', 'score_07_count', 'score_06_count',
                'score_05_count', 'score_04_count', 'score_03_count', 'score_02_count',
                'score_01_count','clubs',  'pics']

'''
Overall columns 
columns = ['anime_id', 'anime_url', 'title', 'synopsis', 'main_pic', 'type',           
                'source_type', 'num_episodes', 'status', 'start_date', 'end_date',
                'season', 'studios', 'genres', 'score', 'score_count', 'score_rank',
                'popularity_rank', 'members_count', 'favorites_count', 'watching_count',
                'completed_count', 'on_hold_count', 'dropped_count',
                'plan_to_watch_count', 'total_count', 'score_10_count',
                'score_09_count', 'score_08_count', 'score_07_count', 'score_06_count',
                'score_05_count', 'score_04_count', 'score_03_count', 'score_02_count',
                'score_01_count', 'clubs', 'pics'],
'''


anime2_df_rev.drop(columns = columns_drop, inplace = True)



### Check data types

Reexamine the data set information to look at data type

In [13]:
anime2_df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13379 entries, 0 to 13378
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   anime_id      13379 non-null  int64  
 1   title         13379 non-null  object 
 2   synopsis      13375 non-null  object 
 3   type          13379 non-null  object 
 4   source_type   13379 non-null  object 
 5   num_episodes  12948 non-null  float64
 6   status        13379 non-null  object 
 7   start_date    13198 non-null  object 
 8   end_date      12799 non-null  object 
 9   studios       10343 non-null  object 
 10  genres        13379 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.1+ MB


Change data type to datatime for date related features

In [14]:
anime2_df_rev['start_date'] = pd.to_datetime(anime2_df_rev['start_date'], errors = 'coerce')
anime2_df_rev['end_date'] = pd.to_datetime(anime2_df_rev['end_date'], errors = 'coerce')


display(anime2_df_rev.head(2))
display(anime2_df_rev.shape)

,anime_id,title,synopsis,type,source_type,num_episodes,status,start_date,end_date,studios,genres
0,2366,Touma Kishinden Oni,Shuramaru is hated and feared by the villagers...,TV,Game,25.0,Finished Airing,1995-10-05,1996-03-21,J.C.Staff,Supernatural
1,4940,Sabaku no Kaizoku! Captain Kuppa,"Sometime in the future, the world was complete...",TV,Manga,26.0,Finished Airing,2001-08-13,2002-02-11,NaN,Action|Adventure


(13379, 11)

#### Revise anime id as index

Anime ID was provided in the dataset. This feature will be checked to confirm if the anime ID's are unique.

In [15]:
#check index
print(f"Anime ID is unique :{len(anime2_df_rev['anime_id'].unique())==anime2_df_rev.shape[0]}")

Anime ID is unique :True


In [16]:
anime2_df_rev.set_index('anime_id',inplace = True)

Anime ID will be used to set value. 

### Review 3rd dataset from Animelist

The third dataset from Animelist will be parsed to look for common keys. Data removal will consider the following:

- Data that is unique to the show will be removed with the exception of the show title. Unique data is considered as traits or information that is not repeated among other entries of data.
- Data that would have been unknown at the time of the show's premiers will also be removed.
- Repetitive data relating to the release or end time of the show will be removed.

### Check header and data

In [17]:
#Create copy... use _rev for copy

anime3_df_rev = anime3_df.copy()

#Review top 3 lines, check shape and columns
display(anime3_df_rev.iloc[:,0:15].head(3))
display(anime3_df_rev.iloc[:,15:].head(3))

display(anime3_df_rev.shape)
display(anime3_df_rev.info())

,anime_id,title,title_english,title_japanese,type,source,episodes,status,airing,aired_from,aired_to,duration,rating,score,scored_by
0,28647,Kappo,NaN,かっぽ,Movie,Original,1.0,Finished Airing,False,2006-01-01T00:00:00+00:00,NaN,4 min,G - All Ages,4.29,413.0
1,35021,Hashire John,NaN,走れジョン,OVA,Original,1.0,Finished Airing,False,1997-01-01T00:00:00+00:00,NaN,24 min,G - All Ages,NaN,NaN
2,30989,Bary-san no Imabari-ben Kouza,NaN,バリィさんのいまばり弁講座,TV,Original,47.0,Finished Airing,False,2015-07-07T00:00:00+00:00,2016-06-03T00:00:00+00:00,30 sec per ep,G - All Ages,NaN,NaN


,rank,popularity,members,favorites,synopsis,background,premiered,broadcast
0,11573.0,11730,716,2,Legs and shoes float across the screen in this...,NaN,NaN,NaN
1,15213.0,17476,96,0,Tatsuya Obayashi rescued a dog from a vacant l...,NaN,NaN,NaN
2,14287.0,12546,520,0,Stars Ehime Prefecture's mascot Barysan.,NaN,Summer 2015,Wednesdays at 02:42 (JST)


(18353, 23)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18353 entries, 0 to 18352
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anime_id        18353 non-null  int64  
 1   title           18353 non-null  object 
 2   title_english   7310 non-null   object 
 3   title_japanese  18292 non-null  object 
 4   type            18353 non-null  object 
 5   source          18353 non-null  object 
 6   episodes        17850 non-null  float64
 7   status          18353 non-null  object 
 8   airing          18353 non-null  bool   
 9   aired_from      18007 non-null  object 
 10  aired_to        8274 non-null   object 
 11  duration        18353 non-null  object 
 12  rating          18353 non-null  object 
 13  score           13052 non-null  float64
 14  scored_by       13052 non-null  float64
 15  rank            16592 non-null  float64
 16  popularity      18353 non-null  int64  
 17  members         18353 non-null 

None

#### Review data and remove unnecessary data

The data was reviewed and judged based on the title of the columns. The following are takeaways:

- Anime url, song and picture are unique to the shows and are not required.
- Scores and related metric will be removed .
- Status,airing, and broadcast will be removed as this is repeated information per aired_from and aired_to. 

In [18]:
columns_drop = [
       'title_japanese',
       'airing', 'status',
       'score', 'scored_by', 'rank', 'popularity',
       'members', 'favorites', 'premiered',
       'broadcast', 'background'
]

'''
['anime_id', 'title', 'title_english', 'title_japanese', 'type',
       'source', 'episodes', 'status', 'airing', 'aired_from', 'aired_to',
       'duration', 'rating', 'score', 'scored_by', 'rank', 'popularity',
       'members', 'favorites', 'synopsis', 'background', 'premiered',
       'broadcast']
'''

anime3_df_rev.drop(columns = columns_drop, inplace = True)


### Check data types

Reexamine the data set information to look at data type

In [19]:
anime3_df_rev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18353 entries, 0 to 18352
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   anime_id       18353 non-null  int64  
 1   title          18353 non-null  object 
 2   title_english  7310 non-null   object 
 3   type           18353 non-null  object 
 4   source         18353 non-null  object 
 5   episodes       17850 non-null  float64
 6   aired_from     18007 non-null  object 
 7   aired_to       8274 non-null   object 
 8   duration       18353 non-null  object 
 9   rating         18353 non-null  object 
 10  synopsis       17187 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 1.5+ MB


Revise data feature to datetime 

In [20]:
anime3_df_rev['aired_from'] = pd.to_datetime(anime3_df_rev['aired_from'], errors = 'coerce')
anime3_df_rev['aired_to'] = pd.to_datetime(anime3_df_rev['aired_to'], errors = 'coerce')

In [21]:
display(anime3_df_rev.head())
display(anime3_df_rev.shape)

,anime_id,title,title_english,type,source,episodes,aired_from,aired_to,duration,rating,synopsis
0,28647,Kappo,NaN,Movie,Original,1.0,2006-01-01 00:00:00+00:00,NaT,4 min,G - All Ages,Legs and shoes float across the screen in this...
1,35021,Hashire John,NaN,OVA,Original,1.0,1997-01-01 00:00:00+00:00,NaT,24 min,G - All Ages,Tatsuya Obayashi rescued a dog from a vacant l...
2,30989,Bary-san no Imabari-ben Kouza,NaN,TV,Original,47.0,2015-07-07 00:00:00+00:00,2016-06-03 00:00:00+00:00,30 sec per ep,G - All Ages,Stars Ehime Prefecture's mascot Barysan.
3,26255,Mina no Bousai Mura Dzukuri,NaN,OVA,Unknown,1.0,2008-01-01 00:00:00+00:00,NaT,20 min,G - All Ages,A typhoon hits Mina's village. As everyone beg...
4,33174,Nori P-chan,NaN,OVA,Manga,1.0,1989-12-25 00:00:00+00:00,NaT,27 min,G - All Ages,An anime adaptation of a shoujo manga of the s...


(18353, 11)

#### Check if index can be replaced by Anime ID

Anime ID was provided in the dataset. This feature will be checked to confirm if the anime ID's are unique.

In [22]:
#check index
print(f"Anime ID is unique :{len(anime3_df_rev['anime_id'].unique())==anime3_df_rev.shape[0]}")

Anime ID is unique :True


In [23]:
anime3_df_rev.set_index('anime_id',inplace = True)

Check that all data sets use the same id system 

In [24]:
#check all anime list are similar using random checks
i = 1500
print(f"For the index {i}, title for datasets are {anime1_df_rev.loc[i,'title']}, \
{anime2_df_rev.loc[i,'title']}, and {anime3_df_rev.loc[i,'title']}, respectively \n")
#check all anime list are similar using random checks
i = 1500*2
print(f"For the index {i}, title for datasets are {anime1_df_rev.loc[i,'title']}, \
{anime2_df_rev.loc[i,'title']}, and {anime3_df_rev.loc[i,'title']}, respectively\n")
#check all anime list are similar using random checks
i = 1500*4
print(f"For the index {i}, title for datasets are {anime1_df_rev.loc[i,'title']}, \
{anime2_df_rev.loc[i,'title']}, and {anime3_df_rev.loc[i,'title']}, respectively")

For the index 1500, title for datasets are Teizokurei Daydream, Teizokurei Daydream, and Teizokurei Daydream, respectively 

For the index 3000, title for datasets are Taiho Shichau zo: Full Throttle, Taiho Shichau zo: Full Throttle, and Taiho Shichau zo: Full Throttle, respectively

For the index 6000, title for datasets are Haruwo, Haruwo, and Haruwo, respectively


Taking 3 samples, the titles are the same. As titles are unique, the index system can be assumed to be the same for all anime list data set. 

## Combine Animelist Datasets

For the Animelist datasets, the data has been condensed based on the metrics noted above. Furthermore, the data has been organized by anime_id. 

All data set will be concatenated by anime_id and index for each date entry row will be revised to anime_id.

#### Concatenate Dataset

In [25]:
# add prefix to col to create unique values

anime2_df_rev = anime2_df_rev.add_prefix("2_")
anime3_df_rev = anime3_df_rev.add_prefix("3_")


In [26]:
#Concat
anime_3_combine = pd.concat([anime1_df_rev,anime2_df_rev,anime3_df_rev,], axis = 1)
anime_3_combine.head()

,title,title_english,title_synonyms,type,source,episodes,status,duration,rating,related,...,3_title,3_title_english,3_type,3_source,3_episodes,3_aired_from,3_aired_to,3_duration,3_rating,3_synopsis
anime_id,,,,,,,,,,,,,,,,,,,,,
1,Cowboy Bebop,Cowboy Bebop,NaN,TV,Original,26.0,Finished Airing,24 min. per ep.,R - 17+ (violence & profanity),"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",...,Cowboy Bebop,Cowboy Bebop,TV,Original,26.0,1998-04-03 00:00:00+00:00,1999-04-24 00:00:00+00:00,24 min per ep,R - 17+ (violence & profanity),"In the year 2071, humanity has colonized sever..."
5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,Movie,Original,1.0,Finished Airing,1 hr. 54 min.,R - 17+ (violence & profanity),"{'Parent story': [{'mal_id': 1, 'type': 'anime...",...,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,Movie,Original,1.0,2001-09-01 00:00:00+00:00,NaT,1 hr 55 min,R - 17+ (violence & profanity),"Another day, another bounty—such is the life o..."
6,Trigun,Trigun,NaN,TV,Manga,26.0,Finished Airing,24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",...,Trigun,Trigun,TV,Manga,26.0,1998-04-01 00:00:00+00:00,1998-09-30 00:00:00+00:00,24 min per ep,PG-13 - Teens 13 or older,"Vash the Stampede is the man with a $$60,000,0..."
7,Witch Hunter Robin,Witch Hunter Robin,WHR,TV,Original,26.0,Finished Airing,25 min. per ep.,PG-13 - Teens 13 or older,[],...,Witch Hunter Robin,Witch Hunter Robin,TV,Original,26.0,2002-07-02 00:00:00+00:00,2002-12-24 00:00:00+00:00,25 min per ep,PG-13 - Teens 13 or older,Witches are individuals with special powers li...
8,Beet the Vandel Buster,Beet the Vandel Buster,"Adventure King Beet, Bouken Ou Beet",TV,Manga,52.0,Finished Airing,23 min. per ep.,PG - Children,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",...,Bouken Ou Beet,Beet the Vandel Buster,TV,Manga,52.0,2004-09-30 00:00:00+00:00,2005-09-29 00:00:00+00:00,23 min per ep,PG - Children,It is the dark century and the people are suff...


#### Check columns and null values

In [27]:
anime_3_combine.columns

Index(['title', 'title_english', 'title_synonyms', 'type', 'source',
       'episodes', 'status', 'duration', 'rating', 'related', 'producer',
       'licensor', 'studio', 'genre', 'aired_start', 'aired_end', '2_title',
       '2_synopsis', '2_type', '2_source_type', '2_num_episodes', '2_status',
       '2_start_date', '2_end_date', '2_studios', '2_genres', '3_title',
       '3_title_english', '3_type', '3_source', '3_episodes', '3_aired_from',
       '3_aired_to', '3_duration', '3_rating', '3_synopsis'],
      dtype='object')

In [28]:
(anime_3_combine.isnull().sum())

title               4654
title_english      13408
title_synonyms     10195
type                4654
source              4654
episodes            4654
status              4654
duration            4654
rating              4654
related             4654
producer           10844
licensor           15759
studio             10588
genre               4718
aired_start         6345
aired_end           6845
2_title             5753
2_synopsis          5757
2_type              5753
2_source_type       5753
2_num_episodes      6184
2_status            5753
2_start_date        5934
2_end_date          6333
2_studios           8789
2_genres            5753
3_title              779
3_title_english    11822
3_type               779
3_source             779
3_episodes          1282
3_aired_from        1125
3_aired_to         10858
3_duration           779
3_rating             779
3_synopsis          1945
dtype: int64

It can be seen there is repeating features (columns) and various columns have missing data. The missing data is likely due to the concatenating various dataset. As various features represent the same data, features from other columns will be used to fill in the missing information and then dropped afterwards. 

The following are features to fill, and features used to fill and then the latter will be drop:
- `title`: filled by `2_title` and`3_title`
- `type`: filled by `2_type` and`3_type`
- `source`: filled by `2_source_type'`and`3_source`
- `status`: filled by `2_status` 
- `duration`: filled by `3_duration`
- `rating`: filled by `3_rating`
- `episodes`: filled by `2_num_episodes` and`3_episodes`
- `studio`: filled by `2_studio`
- `genre`: filled by `2_genre`
- `aired_start`: filled by `2_start_date` and `3_aired_from`
- `aired_end`: filled by `2_end_date` and`3_aired_to`
- `2_synopsis`: filled by `3_synopsis` 

#### Fill na from other data set and drop duplicate column

In [29]:
# title 
anime_3_combine['title'].fillna(anime_3_combine['2_title'], inplace=True)
anime_3_combine['title'].fillna(anime_3_combine['3_title'], inplace=True)

# type 
anime_3_combine['type'].fillna(anime_3_combine['2_type'], inplace=True)
anime_3_combine['type'].fillna(anime_3_combine['3_type'], inplace=True)

# source 
anime_3_combine['source'].fillna(anime_3_combine['2_source_type'], inplace=True)
anime_3_combine['source'].fillna(anime_3_combine['3_source'], inplace=True)

# status 
anime_3_combine['status'].fillna(anime_3_combine['2_status'], inplace=True)

# duration            
anime_3_combine['duration'].fillna(anime_3_combine['3_duration'], inplace=True)

# rating  
anime_3_combine['rating'].fillna(anime_3_combine['3_rating'], inplace=True)

# episodes 
anime_3_combine['episodes'].fillna(anime_3_combine['2_num_episodes'], inplace=True)
anime_3_combine['episodes'].fillna(anime_3_combine['3_episodes'], inplace=True)

# studio 
anime_3_combine['studio'].fillna(anime_3_combine['2_studios'], inplace=True)

# genre 
anime_3_combine['genre'].fillna(anime_3_combine['2_genres'], inplace=True)
 
# aired_start 
anime_3_combine['aired_start'].fillna(anime_3_combine['2_start_date'], inplace=True)
anime_3_combine['aired_start'].fillna(anime_3_combine['3_aired_from'], inplace=True)

# aired_end 
anime_3_combine['aired_end'].fillna(anime_3_combine['2_end_date'], inplace=True)
anime_3_combine['aired_end'].fillna(anime_3_combine['3_aired_to'], inplace=True)

# synopsis          
anime_3_combine['2_synopsis'].fillna(anime_3_combine['3_synopsis'], inplace=True)


#droping column 

drop_columns = [
    
'2_title', '2_type', '2_source_type',
'2_num_episodes', '2_status', '2_start_date', '2_end_date', '2_studios',
'2_genres', '3_title', '3_type', '3_source', '3_episodes',
'3_aired_from', '3_aired_to', '3_duration', '3_rating'

]




'''
Index(['title', 'type', 'source', 'episodes', 'status', 'duration', 'rating',
       'related', 'producer', 'licensor', 'studio', 'genre', 'aired_start',
       'aired_end', '2_title', '2_synopsis', '2_type', '2_source_type',
       '2_num_episodes', '2_status', '2_start_date', '2_end_date', '2_studios',
       '2_genres', '3_title', '3_type', '3_source', '3_episodes',
       '3_aired_from', '3_aired_to', '3_duration', '3_rating', '3_synopsis'],
      dtype='object'
'''

anime_3_combine.drop(columns = drop_columns, inplace=True)




#### Check new columns and null values

In [30]:
anime_3_combine.columns

Index(['title', 'title_english', 'title_synonyms', 'type', 'source',
       'episodes', 'status', 'duration', 'rating', 'related', 'producer',
       'licensor', 'studio', 'genre', 'aired_start', 'aired_end', '2_synopsis',
       '3_title_english', '3_synopsis'],
      dtype='object')

In [31]:
(anime_3_combine.isnull().sum())/anime_3_combine.shape[0]*100

title               0.000000
title_english      70.081539
title_synonyms     53.287686
type                0.000000
source              0.000000
episodes            2.179594
status              6.826260
duration            3.826051
rating              3.826051
related            24.325737
producer           56.679908
licensor           82.369852
studio             38.004391
genre               7.092829
aired_start         1.855530
aired_end          14.436546
2_synopsis          3.240644
3_title_english    61.791762
3_synopsis         10.166214
dtype: float64

These columns appear to require further exploration during EDA:
- 'producer'
- 'licensor'
- 'related'
- 'studio'

## Dataset for TV revenue

### Review dataset from someanithing

The data set  was taken from someanithing.com. The data contains information of the revenue for anime TV shows. From the data source, the revenue was taken as yen. 

The data will be reviewed for any other information and cleaned. There will be further steps to change titles to match that of the combined dataset of the Animelist. 

In [32]:
#Create copy of revenue dataframe
anime_1rev_df_rev = anime_1rev_df.copy()

In [33]:
# Check header shape and information
display(anime_1rev_df_rev.head())
display(anime_1rev_df_rev.shape)
display(anime_1rev_df_rev.info(10))

,Series Title,Type,Season,Year,"Sales Avg, Initial Release","Sales Avg, Re-releases","Sales Avg, Total",Revenue\n(Initial Release),Source Material,Animation Studio,Publisher,Distributor,万円
0,.hack//SIGN,TV,Spring,2002,6032,0,6032,362220000,game,Bee Train,Bandai Visual,Bandai Visual,36222
1,.hack//Tasogare no Udewa Densetsu,TV,Winter,2003,2954,0,2954,102810000,game,Bee Train,Bandai Visual,Bandai Visual,10281
2,.hack//Roots,TV,Spring,2006,2138,0,2138,113000000,game,Bee Train,Bandai Visual,Bandai Visual,11300
3,009-1,TV,Fall,2006,1258,0,1258,46390000,manga,Ishimori Entertainment,Aniplex,Sony,4639
4,07-Ghost,TV,Spring,2009,2529,0,2529,197230000,manga,Studio Deen,Avex,Avex,19723


(2450, 13)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2450 entries, 0 to 2449
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Series Title                2450 non-null   object
 1   Type                        2450 non-null   object
 2   Season                      2450 non-null   object
 3   Year                        2450 non-null   int64 
 4   Sales Avg, Initial Release  2450 non-null   int64 
 5   Sales Avg, Re-releases      2450 non-null   int64 
 6   Sales Avg, Total            2450 non-null   int64 
 7   Revenue
(Initial Release)   2450 non-null   int64 
 8   Source Material             2450 non-null   object
 9   Animation Studio            2450 non-null   object
 10  Publisher                   2450 non-null   object
 11  Distributor                 2450 non-null   object
 12  万円                          2450 non-null   int64 
dtypes: int64(6), object(7)
memory usage: 249.0+ KB


None

#### Check duplication 
Look for duplicate and remove any features or rows that are duplicated

In [34]:
print(f'There are {anime_1rev_df_rev.duplicated().sum()} duplicated rows')

There are 2 duplicated rows


In [35]:
#remove duplicates
anime_1rev_df_rev.drop_duplicates(inplace=True)
print('Duplicated rows have been removed')

Duplicated rows have been removed


#### Drop unnecessary columns

Key information that may be used are:
- Title
- Type
- Year
- Revenue
- Source Material
- Studio
- Publisher
- Distributor

There is additional information regarding the breakdown of the sales and the seasons. These information will be removed.

In [36]:
#drop unnecessary 

anime_1rev_df_rev.drop(columns = [
    'Sales Avg, Initial Release','Season',
       'Sales Avg, Re-releases', 'Sales Avg, Total','万円'
], inplace =True)

anime_1rev_df_rev.head()

,Series Title,Type,Year,Revenue\n(Initial Release),Source Material,Animation Studio,Publisher,Distributor
0,.hack//SIGN,TV,2002,362220000,game,Bee Train,Bandai Visual,Bandai Visual
1,.hack//Tasogare no Udewa Densetsu,TV,2003,102810000,game,Bee Train,Bandai Visual,Bandai Visual
2,.hack//Roots,TV,2006,113000000,game,Bee Train,Bandai Visual,Bandai Visual
3,009-1,TV,2006,46390000,manga,Ishimori Entertainment,Aniplex,Sony
4,07-Ghost,TV,2009,197230000,manga,Studio Deen,Avex,Avex


### Change Data for revenue to USD

Revenue information will be revised to match USD value. The dollar ratio will use a 0.0075 yen to 1 USD. 

In [37]:
# value is in yen --> convert to USD : 0.0075 Yen = 1 USD

#Remove special character
anime_3_combine['title'] = anime_3_combine['title'].str.replace('[^a-zA-Z0-9/.\-! ]+', ' ', regex=True)
anime_1rev_df_rev['Revenue\n(Initial Release)'] = anime_1rev_df_rev['Revenue\n(Initial Release)']*0.0075 

anime_1rev_df_rev.sort_values('Series Title').head(10)

,Series Title,Type,Year,Revenue\n(Initial Release),Source Material,Animation Studio,Publisher,Distributor
2,.hack//Roots,TV,2006,847500.0,game,Bee Train,Bandai Visual,Bandai Visual
0,.hack//SIGN,TV,2002,2716650.0,game,Bee Train,Bandai Visual,Bandai Visual
1,.hack//Tasogare no Udewa Densetsu,TV,2003,771075.0,game,Bee Train,Bandai Visual,Bandai Visual
3,009-1,TV,2006,347925.0,manga,Ishimori Entertainment,Aniplex,Sony
4,07-Ghost,TV,2009,1479225.0,manga,Studio Deen,Avex,Avex
5,100-man no Inochi no Ue ni Ore wa Tatteiru [Se...,TV,2020,0.0,manga,Maho Film,Warner Brothers,NBC Universal (Geneon)
6,11eyes,TV,2009,137700.0,visual novel,Douga Koubou,Marvelous AQL,Pony Canyon
7,18if,TV,2017,0.0,game,Gonzo,Gonzo,Gonzo
8,2.43: Seiin Koukou Danshi Volley-bu,TV,2021,54150.0,novel,David Production,Aniplex,Sony
9,22/7,TV,2020,908775.0,original,A1 Pictures,Aniplex,Sony


### Revise title to match that in the combined data set

The title name of the revenue dataset from someanithing will be revised to match that of the Animelist dataset. The criteria will be based on the following:

1) Remove all special characters (non alphabetic and non numeric) except period, dash, and slash. Match those that are complete match.

2) Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match. 

3) Split titles into lists, look for 90% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match.

4) Split titles into lists, look for 75% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match.

5) Split titles into lists, look for 75% match with only alphabetic character based on revenue title and match those with only 1 potential match.

6) Check for word for word match while removing non alphabet character.

#### First, turn all titles into lower case and remove non-alphabetic and non-numeric character except space, period, slash, and dash

In [40]:
#turn text to lower case
anime_1rev_df_rev['Series Title']=anime_1rev_df_rev['Series Title'].str.lower()
anime_3_combine['title'] = anime_3_combine['title'].str.lower()

#Remove special character
anime_3_combine['title'] = anime_3_combine['title'].str.replace('[^a-zA-Z0-9/.\- ]+', ' ', regex=True)
anime_1rev_df_rev['Series Title'] = anime_1rev_df_rev['Series Title'].str.replace('[^a-zA-Z0-9/.\- ]+', ' ', regex=True)

In [41]:
def not_movie_title_comparison(df1,title1,df2,title2,type2):
    """
    Input: two dataframe, title names in dataframes, and type name for dataframe 2
    
    Output: list of titles for each dataframe, 1 list of movies with same title both frames, and 2 list of movie that are not
            included in the other dataframe
    """
    title1_list = set(pd.unique(df1[title1]))

    title2_list = set(pd.unique(df2[df2[type2]!='Movie'][title2]))

    title1_title2_same =(title1_list.intersection(title2_list))
    title1_title2_dif =(title1_list.difference(title2_list))
    title2_title1_dif = (title2_list.difference(title1_list))
    
    return title1_list, title2_list, title1_title2_same, title1_title2_dif, title2_title1_dif

#### 1) Check for complete match

In [42]:
# movie title comparison output as list
title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 1695.
The number of shows that need to be matched: 736.


#### 2) Check for 90% match


 Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [43]:
#Cycle for words 90% 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif in show_combine_diff:
    
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title in title_list_combine:
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1rev_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1rev_df_rev.loc[anime_1rev_df_rev['Series Title'] == key,'Series Title'] = dictionary[key][0]     

In [44]:
# movie title comparison output as list
title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 1862.
The number of shows that need to be matched: 566.


#### 3) Check for 90% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [45]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif1 in show_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1rev_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1rev_df_rev.loc[anime_1rev_df_rev['Series Title'] == key,'Series Title'] = dictionary[key][0]     

In [46]:
# movie title comparison output as list
title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 1898.
The number of shows that need to be matched: 530.


#### 4) Check for 75% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 75% match based on revenue title and match those with only 1 potential match

In [47]:
#Cycle for words 75% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif1 in show_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
        #Remove special character and add space
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1rev_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.75, then add title to list 
            if counter/ length_word > 0.75:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1rev_df_rev.loc[anime_1rev_df_rev['Series Title'] == key,'Series Title'] = dictionary[key][0]     

In [48]:
# movie title comparison output as list
title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 1926.
The number of shows that need to be matched: 499.


#### 5) Check for 90% match alphabetic character


 Remove non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [49]:
#Cycle for words 75% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 
 
for title_dif in show_combine_diff:

    #Remove special character and numbers and add space
    title_dif1 = re.sub(r'[^A-Za-z ]+', ' ', title_dif)
    split_word = (title_dif1.split())
    length_word = len(split_word)
    list_ = []
    
    for title in combine_show_diff:
        
        #Remove special character and numbers and add space
        title1 = re.sub(r'[^A-Za-z ]+', ' ', title)
        if any(word in title1 for word in split_word):
            counter =0
            for word in split_word:
                if word in title1:
                    counter += 1 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
            
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary

for key in dictionary:
  
    if len(dictionary[key]) == 1:
        anime_1rev_df_rev.loc[anime_1rev_df_rev['Series Title'] == key,'Series Title'] = dictionary[key][0]     
                
                
        

In [50]:
# movie title comparison output as list
title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 1943.
The number of shows that need to be matched: 481.


#### 6) Check for word for word match keeping only alphabet character


 Remove non alphabetic, look for 100% matches, and match those with only 1 potential match

In [51]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif1 in show_combine_diff:
    
    #Remove everything but alphabet character
    title_dif = re.sub(r'[^A-Za-z]+', '', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
         #Remove everything but alphabet character
        title = re.sub(r'[^A-Za-z]+', '', title1)
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if title == title_dif:
            list_.append(title1)

            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1rev_df_rev.loc[anime_1rev_df_rev['Series Title'] == key,'Series Title'] = dictionary[key][0]     

In [52]:
# movie title comparison output as list

title_list_show,title_list_combine,show_combine_same,show_combine_diff,combine_show_diff = not_movie_title_comparison(anime_1rev_df_rev,'Series Title',anime_3_combine,'title','type')

print(f'The number of shows that have the same titles is: {len(show_combine_same)}.')
print(f'The number of shows that need to be matched: {len(show_combine_diff)}.')

The number of shows that have the same titles is: 2118.
The number of shows that need to be matched: 303.


### Combine Animelist and Someanithing (TV) together

Merge all tables using left join. Use anime_id based on titles as common key.

In [53]:
# Create df for concate by using id to avoid duplicates 
condition_1 = anime_3_combine['type']!='Movie'
condition_2 = ~anime_3_combine['title'].duplicated(keep=False)

not_movie_not_duplicated_anime_df = anime_3_combine[(condition_1 & condition_2)].copy()

# create a dictionary for title to anime id

dictionary_anime_id_shows={}

    #Loop to fill dictionary
for i,row in not_movie_not_duplicated_anime_df.iterrows():
    dictionary_anime_id_shows[row['title']] = i


#Create empty dataframe to fill with corrected index and left over to manual observe 
    
concated_tv_df = pd.DataFrame(columns =anime_1rev_df_rev.columns )
leftover_tv_df = pd.DataFrame(columns =anime_1rev_df_rev.columns )

# loop through show dataframe to change index to matching index
    #use title for condition to change and add to new dataframe
unique_titles = dictionary.keys()
for i, row in anime_1rev_df_rev.iterrows():
    unique_titles = dictionary_anime_id_shows.keys()
    if row['Series Title'] in unique_titles:
        
        new_index = dictionary_anime_id_shows[row['Series Title']]
        concated_tv_df.loc[new_index,:] = row
        
        dictionary_anime_id_shows.pop(row['Series Title'])
    else:
        leftover_tv_df.loc[len(leftover_tv_df.index),:] = row

In [54]:
### Merge data together 
anime_rev_combine = pd.merge(anime_3_combine,concated_tv_df,how= 'left', left_index=True, right_index=True)

In [55]:
#Check top 5 data 
anime_rev_combine.head()

,title,title_english,title_synonyms,type,source,episodes,status,duration,rating,related,...,3_title_english,3_synopsis,Series Title,Type,Year,Revenue\n(Initial Release),Source Material,Animation Studio,Publisher,Distributor
anime_id,,,,,,,,,,,,,,,,,,,,,
1,cowboy bebop,Cowboy Bebop,NaN,TV,Original,26.0,Finished Airing,24 min. per ep.,R - 17+ (violence & profanity),"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",...,Cowboy Bebop,"In the year 2071, humanity has colonized sever...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cowboy bebop tengoku no tobira,Cowboy Bebop: The Movie,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,Movie,Original,1.0,Finished Airing,1 hr. 54 min.,R - 17+ (violence & profanity),"{'Parent story': [{'mal_id': 1, 'type': 'anime...",...,Cowboy Bebop: The Movie,"Another day, another bounty—such is the life o...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,trigun,Trigun,NaN,TV,Manga,26.0,Finished Airing,24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",...,Trigun,"Vash the Stampede is the man with a $$60,000,0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,witch hunter robin,Witch Hunter Robin,WHR,TV,Original,26.0,Finished Airing,25 min. per ep.,PG-13 - Teens 13 or older,[],...,Witch Hunter Robin,Witches are individuals with special powers li...,witch hunter robin,TV,2002,1186650.0,original,Sunrise,Bandai Visual,Bandai Visual
8,beet the vandel buster,Beet the Vandel Buster,"Adventure King Beet, Bouken Ou Beet",TV,Manga,52.0,Finished Airing,23 min. per ep.,PG - Children,"{'Adaptation': [{'mal_id': 1348, 'type': 'mang...",...,Beet the Vandel Buster,It is the dark century and the people are suff...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Financial table film

### Review dataset from Someanithing

The data set  was taken from someanithing.com and specifically, the data contains revenue information for anime movies. From the data source, the revenue was taken as yen. 

The data will be reviewed for any other information and cleaned. There will be further steps to change titles to match that of the combined dataset of the anime list. 

In [56]:
#copy and add _rev to data
anime_1film_df_rev=anime_1film_df.copy()

In [57]:
#display head, shape and info
display(anime_1film_df_rev.head(3))
display(anime_1film_df_rev.shape)
display(anime_1film_df_rev.info())

,Movie Title,Type,Gross,Weeks,Screens,Max Screens,Release Date
0,Accel World: Infinite Burst,Late Night,"¥115,000,000",NaN,19,NaN,2016-07-23
1,Aikatsu Movie,General,"¥172,727,250",1,150,NaN,2014-12-13
2,Akira (IMAX 2020),General,"¥106,389,400",12,NaN,NaN,2020-04-03


(485, 7)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 485 entries, 0 to 484
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Movie Title   485 non-null    object        
 1   Type          485 non-null    object        
 2   Gross         472 non-null    object        
 3   Weeks         322 non-null    object        
 4   Screens       283 non-null    object        
 5   Max Screens   47 non-null     float64       
 6   Release Date  485 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 26.6+ KB


None

#### Drop unnecessary features

The data for the table contains the following:

- title
- type
- Gross
- Weeks on movie theater
- Screens (number of screens)
- Max Screens
- Release Date

Feature 'weeks' and 'screens' will be removed as these statistics are only present in the dataset for film and would skew data if kept. 

In [58]:
#drop unnecessary 

anime_1film_df_rev.drop(columns = [

     'Weeks', 'Screens', 'Max Screens'

], inplace =True)


'''

Index(['Movie Title', 'Type', 'Gross', 'Weeks', 'Screens', 'Max Screens',
       'Release Date'],
      dtype='object')

'''
#check
anime_1film_df_rev.head()

,Movie Title,Type,Gross,Release Date
0,Accel World: Infinite Burst,Late Night,"¥115,000,000",2016-07-23
1,Aikatsu Movie,General,"¥172,727,250",2014-12-13
2,Akira (IMAX 2020),General,"¥106,389,400",2020-04-03
3,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,Late Night,"¥1,040,000,000",2013-08-31
4,Ao no Exorcist Movie,Late Night,"¥560,000,000",2013-12-28


#### Revise Gross data

Gross data is provided as yen. 

For `Gross` feature, special characters will be removed and value will be changed to dollar based on a 0.0075:1 conversion rate. 

In [59]:
#remove special characters
anime_1film_df_rev['Gross'] = anime_1film_df_rev[~anime_1film_df_rev['Gross'].isna()]['Gross'].map(lambda x: x.replace('¥',""))
anime_1film_df_rev['Gross'] = anime_1film_df_rev[~anime_1film_df_rev['Gross'].isna()]['Gross'].map(lambda x: x.replace(',',""))

#turn value to float
anime_1film_df_rev['Gross']=anime_1film_df_rev['Gross'].astype('float64')

# value is in yen --> convert to USD : 0.0075 Yen = 1 USD
#conversion of the gross
anime_1film_df_rev['Gross'] = anime_1film_df_rev['Gross']*0.0075

In [60]:
#check
anime_1film_df_rev.head()

,Movie Title,Type,Gross,Release Date
0,Accel World: Infinite Burst,Late Night,862500.000,2016-07-23
1,Aikatsu Movie,General,1295454.375,2014-12-13
2,Akira (IMAX 2020),General,797920.500,2020-04-03
3,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,Late Night,7800000.000,2013-08-31
4,Ao no Exorcist Movie,Late Night,4200000.000,2013-12-28


### Revise movie title to match that in the combined data set

The title name of the revenue dataset `Movie Title` will be revised to match that of the Animelist dataset. The criteria will be based on the following:

1) Remove all special characters (non alphabetic and non numeric) except period, dash, and slash. Match those that are complete match

2) Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match 

3) Split titles into lists, look for 90% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match

4) Split titles into lists, look for 75% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match

5) Split titles into lists, look for 75% match with only alphabetic character based on revenue title and match those with only 1 potential match

6) Check for word for word match while removing non alphabet character

7) Check for 66% match with numeric and alphabetic character and words counts of potential matches does not exceed word count of film



#### First, turn all titles into lower case and remove non-alphabetic and non-numeric character except space, period, slash, and dash

In [61]:
#turn text to lower case
anime_1film_df_rev['Movie Title']=anime_1film_df_rev['Movie Title'].str.lower()
anime_3_combine['title'] = anime_3_combine['title'].str.lower()

#Remove special character
anime_3_combine['title'] = anime_3_combine['title'].str.replace('[^a-zA-Z0-9/.\- ]+', ' ', regex=True)
anime_1film_df_rev['Movie Title'] = anime_1film_df_rev['Movie Title'].str.replace('[^a-zA-Z0-9/.\- ]+', ' ', regex=True)

In [62]:
def movie_title_comparison(df1,title1,df2,title2,type2):
    """
    Input: two dataframe, title names in dataframes, and type name for dataframe 2
    
    Output: list of titles for each dataframe, 1 list of movies with same title both frames, and 2 list of movie that are not
            included in the other dataframe
    """
    title1_list = set(pd.unique(df1[title1]))

    title2_list = set(pd.unique(df2[df2[type2]=='Movie'][title2]))

    title1_title2_same =(title1_list.intersection(title2_list))
    title1_title2_dif =(title1_list.difference(title2_list))
    title2_title1_dif = (title2_list.difference(title1_list))
    
    return title1_list, title2_list, title1_title2_same, title1_title2_dif, title2_title1_dif

#### 1) Check for complete match

In [63]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 106.
The number of film that need to be matched: 378.


#### 2) Check for 90% match


 Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [64]:
#Cycle for words 90% 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif in film_combine_diff:
    
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    #loop through titles from animelist that are different 
    for title in title_list_combine:
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]     

In [65]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 277.
The number of film that need to be matched: 206.


#### 3) Check for 90% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [66]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]         

In [67]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 288.
The number of film that need to be matched: 195.


#### 4) Check for 75% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 75% match based on revenue title and match those with only 1 potential match

In [68]:
#Cycle for words 75% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
        #Remove special character and add space
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.75, then add title to list 
            if counter/ length_word > 0.75:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]         

In [69]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 365.
The number of film that need to be matched: 114.


#### 5) Check for 90% match alphabetic character


 Remove non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [70]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 
 
for title_dif in film_combine_diff:

    #Remove special character and numbers and add space
    title_dif1 = re.sub(r'[^A-Za-z ]+', ' ', title_dif)
    split_word = (title_dif1.split())
    length_word = len(split_word)
    list_ = []
    
    for title in combine_show_diff:
        
        #Remove special character and numbers and add space
        title1 = re.sub(r'[^A-Za-z ]+', ' ', title)
        if any(word in title1 for word in split_word):
            counter =0
            for word in split_word:
                if word in title1:
                    counter += 1 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
            
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary

for key in dictionary:
  
    if len(dictionary[key]) == 1:
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]                         
                
        

In [71]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 365.
The number of film that need to be matched: 114.


#### 6) Check for word for word match keeping only alphabet character


 Remove non alphabetic, look for 100% matches, and match those with only 1 potential match

In [72]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove everything but alphabet character
    title_dif = re.sub(r'[^A-Za-z]+', '', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
         #Remove everything but alphabet character
        title = re.sub(r'[^A-Za-z]+', '', title1)
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if title == title_dif:
            list_.append(title1)

            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]                         


In [73]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 367.
The number of film that need to be matched: 112.


#### 7) Check for 66% match with numeric and alphabetic character and words counts of potential matches does not exceed word count of film


Remove nonnumeric and non alphabetic, split titles into lists, and look for 66% match based on revenue title while limiting total count of word in animelist title to total count of revenue title. Lastly match those with only 1 potential match

In [74]:
#Cycle for words 66% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
        #Remove special character and add space
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        #split words into list 
        split_title = (title.split())
    
        #Get number of words in list to compare
        length_word_title = len(split_title)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.66, then add title to list 
            if counter/ length_word > 0.66 and length_word_title/length_word< 1.0:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        anime_1film_df_rev.loc[anime_1film_df_rev['Movie Title'] == key,'Movie Title'] = dictionary[key][0]         

In [75]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = movie_title_comparison(anime_1film_df_rev,'Movie Title',anime_3_combine,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 377.
The number of film that need to be matched: 97.


### Combine all explored datasets and someanithing (film)

Dataset will be merged all tables using left join and titles as common key.

In [76]:
# Create df for concate by using id to avoid duplicates 
condition_1 = anime_3_combine['type']!='TV'
condition_2 = ~anime_3_combine['title'].duplicated(keep=False)

not_show_not_duplicated_anime_df = anime_3_combine[(condition_1 & condition_2)].copy()

# create a dictionary for title to anime id

dictionary_anime_id_film={}

    #Loop to fill dictionary
for i,row in not_show_not_duplicated_anime_df.iterrows():
    dictionary_anime_id_film[row['title']] = i


#Create empty dataframe to fill with corrected index and left over to manual observe 
    
concated_film_df = pd.DataFrame(columns =anime_1film_df_rev.columns )
leftover_film_df = pd.DataFrame(columns =anime_1film_df_rev.columns )


# loop through show dataframe to change index to matching index
    #use title for condition to change and add to new dataframe
unique_titles = dictionary.keys()
for i, row in anime_1film_df_rev.iterrows():
    unique_titles = dictionary_anime_id_film.keys()
    if row['Movie Title'] in unique_titles:
 
        new_index = dictionary_anime_id_film[row['Movie Title']]
        concated_film_df.loc[new_index,:] = row
        
        dictionary_anime_id_film.pop(row['Movie Title'])
    else:
        leftover_film_df.loc[len(leftover_film_df.index),:] = row

In [77]:
# Merged data using left join based on anime_rev_combine and use title feature as key
anime_final_combine = pd.merge(anime_rev_combine,concated_film_df,how= 'left', left_index=True, right_index=True)

In [78]:
#Check first 3 data and info for column names
display(anime_final_combine.head(3))
display(anime_final_combine.info())

,title,title_english,title_synonyms,type,source,episodes,status,duration,rating,related,...,Year,Revenue\n(Initial Release),Source Material,Animation Studio,Publisher,Distributor,Movie Title,Type_y,Gross,Release Date
anime_id,,,,,,,,,,,,,,,,,,,,,
1,cowboy bebop,Cowboy Bebop,NaN,TV,Original,26.0,Finished Airing,24 min. per ep.,R - 17+ (violence & profanity),"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cowboy bebop tengoku no tobira,Cowboy Bebop: The Movie,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,Movie,Original,1.0,Finished Airing,1 hr. 54 min.,R - 17+ (violence & profanity),"{'Parent story': [{'mal_id': 1, 'type': 'anime...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,trigun,Trigun,NaN,TV,Manga,26.0,Finished Airing,24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19132 entries, 1 to 51333
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   title                      19132 non-null  object 
 1   title_english              5724 non-null   object 
 2   title_synonyms             8937 non-null   object 
 3   type                       19132 non-null  object 
 4   source                     19132 non-null  object 
 5   episodes                   18715 non-null  float64
 6   status                     17826 non-null  object 
 7   duration                   18400 non-null  object 
 8   rating                     18400 non-null  object 
 9   related                    14478 non-null  object 
 10  producer                   8288 non-null   object 
 11  licensor                   3373 non-null   object 
 12  studio                     11861 non-null  object 
 13  genre                      17775 non-null  obj

None

There are various features about revenue data. This information needs to be condensed to a common feature called `revenue`.

In [79]:
# create new feature 'revenue' with np.nan as filled value
anime_final_combine.insert(16, "revenue", np.nan , True)

In [80]:
# fill value in revenue with data from 'Revenue\n(Initial Release)'
anime_final_combine.loc[:,['revenue']] = anime_final_combine.loc[:,['Revenue\n(Initial Release)']].values

# fill missing value in revenue with data from 'Gross'
anime_final_combine.loc[anime_final_combine['revenue'].isna(),['revenue']] =\
    anime_final_combine.loc[anime_final_combine['revenue'].isna(),['Gross']].values

#Drop all 'Revenue\n(Initial Release)' and 'Gross'
anime_final_combine.drop(columns = ['Revenue\n(Initial Release)','Gross'], inplace =True)

In [81]:
# Turn all zero data to np.nan
anime_final_combine.loc[anime_final_combine['revenue']==0,'revenue'] =np.nan

### Review dataset from webscraping 

The data set  was taken from various source that contained information about the revenue for anime movies. From the data source, the revenue was taken as yen. 

The data will be reviewed for any other information and cleaned. There will be further steps to change titles to match that of the combined dataset of the anime list. 

In [82]:
#copy dataset and add _rev

#dataset 6 - film revenue
anime_2film_df_rev =anime_2film_df.copy()

#dataset 7 - film revenue
anime_3film_df_rev = anime_3film_df.copy()

#dataset 8 - film revenue
anime_4film_df_rev = anime_4film_df.copy()

In [83]:
#check data header
anime_2film_df_rev.head()

,0,Ranking,Release Month,Film Title,Gross (estimated) (in billiion Yen),Distributor
0,1,1,Nov.,HOWL'S MOVING CASTLE,20.00,Toho
1,2,2,May,"CRYING OUT LOVE, IN THE CENTER OF THE WORLD",8.50,Toho
2,3,3,Oct.,BE WITH YOU,4.80,Toho
3,4,4,July,POKEMON: DESTINY DEOXYS,4.38,Toho
4,5,5,March,DORAEMON : Nobita in the Wan-Nyan Space-time O...,3.05,Toho


In [84]:
#check data header
anime_3film_df_rev.head()

,Unnamed: 0,Release Date,Title,Production Budget,Opening Weekend,Domestic Box Office,Worldwide Box Office
0,0,"Aug 19, 2022",Doragon boru supa supa hiro…,NaN,NaN,NaN,"$5,000,000"
1,1,"Mar 18, 2022",Gekijouban Jujutsu Kaisen 0,NaN,"$18,003,204","$33,919,605","$189,180,372"
2,2,"Jan 14, 2022",Ryu to sobakasu no hime (竜と…,NaN,"$1,565,658","$4,018,313","$61,012,245"
3,3,"Dec 30, 2021",Eiga Entotsu Machi no Poupelle,NaN,NaN,NaN,"$20,643,297"
4,4,"Nov 24, 2021",Le sommet des dieux,NaN,NaN,NaN,NaN


In [85]:
#check data header
anime_4film_df_rev.head()

,Unnamed: 0,Title,Worldwide gross,Year,Format,Ref.
0,0,Demon Slayer the Movie: Mugen Train,"$505,582,795",2020,Anime,[3]
1,1,Spirited Away,"$395,580,000",2001,Anime,[4]
2,2,Your Name,"$380,140,500",2016,Anime,[4]
3,3,Howl's Moving Castle,"$236,214,446",2004,Anime,[5]
4,4,Ponyo,"$204,826,668",2008,Anime,[6]


Keep Title and Revenue from all dataset

In [86]:
# Revise columns names for title and revenue to keep all information in common collumns

anime_2film_df_rev=anime_2film_df_rev.rename(columns = {'Film Title':'film_title','Gross (estimated)  (in billiion Yen)':'film_revenue'})
anime_3film_df_rev=anime_3film_df_rev.rename(columns = {'Title':'film_title','Worldwide Box Office':'film_revenue'})
anime_4film_df_rev=anime_4film_df_rev.rename(columns = {'Title':'film_title','Worldwide gross':'film_revenue'})

#Revise Data to remove non numeric in revenue data
anime_3film_df_rev['film_revenue']=anime_3film_df_rev['film_revenue'].str.replace('[^0-9]+', '', regex=True)
anime_4film_df_rev['film_revenue']=anime_4film_df_rev['film_revenue'].str.replace('[^0-9]+', '', regex=True)

# Change data type  to numeric
anime_3film_df_rev['film_revenue']=pd.to_numeric(anime_3film_df_rev['film_revenue'],errors='coerce')
anime_4film_df_rev['film_revenue']=pd.to_numeric(anime_4film_df_rev['film_revenue'],errors='coerce')


# data was given in billon of yens. converted to usd
anime_2film_df_rev['film_revenue'] = pd.to_numeric(anime_2film_df_rev['film_revenue'],errors='coerce')*10**9*0.0075


Concat web scrapping into one dataframe

In [87]:
#concat webscraped data
websrapped_df = pd.concat([anime_2film_df_rev[['film_title','film_revenue']],\
                          anime_3film_df_rev[['film_title','film_revenue']],anime_4film_df_rev[['film_title','film_revenue']]])

#Reset index
websrapped_df.reset_index(drop=True,inplace=True)

In [88]:
#Drop all na values and turn title to lower case
websrapped_df.dropna(inplace=True)
websrapped_df['film_title']=websrapped_df['film_title'].str.lower()

In [89]:
# drop duplicated and keep the highest valued revenue
websrapped_df=websrapped_df.sort_values('film_revenue').drop_duplicates('film_title', keep='last')

### Revise movie title  (webscraped) to match that in the combined Animelist data set

The title name of the revenue dataset `Movie Title` will be revised to match that of the Animelist dataset. The criteria will be based on the following:

1) Remove all special characters (non alphabetic and non numeric) except period, dash, and slash. Match those that are complete match

2) Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match 

3) Split titles into lists, look for 90% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match

4) Split titles into lists, look for 75% match with numeric and alphabetic character based on revenue title and match those with only 1 potential match

5) Split titles into lists, look for 75% match with only alphabetic character based on revenue title and match those with only 1 potential match

6) Check for word for word match while removing non alphabet character

7) Check for 66% match with numeric and alphabetic character and words counts of potential matches does not exceed word count of film



#### First, turn all titles into lower case and remove non-alphabetic and non-numeric character except space, period, slash, and dash

In [90]:
#turn text to lower case
websrapped_df['film_title']=websrapped_df['film_title'].str.lower()

#Remove special character

websrapped_df['film_title'] = websrapped_df['film_title'].str.replace('[^a-zA-Z0-9/.\- ]+', ' ', regex=True)

In [91]:
# Create a dataframe for unmatched title from animelist

combined_leftover_df = anime_final_combine[anime_final_combine['revenue'].isna()].copy()


In [92]:
def not_show_title_comparison(df1,title1,df2,title2,type2):
    """
    Input: two dataframe, title names in dataframes, and type name for dataframe 2
    
    Output: list of titles for each dataframe, 1 list of movies with same title both frames, and 2 list of movie that are not
            included in the other dataframe
    """
    title1_list = set(pd.unique(df1[title1]))

    title2_list = set(pd.unique(df2[df2[type2]!='TV'][title2]))

    title1_title2_same =(title1_list.intersection(title2_list))
    title1_title2_dif =(title1_list.difference(title2_list))
    title2_title1_dif = (title2_list.difference(title1_list))
    
    return title1_list, title2_list, title1_title2_same, title1_title2_dif, title2_title1_dif

#### 1) Check for complete match

In [93]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 18.
The number of film that need to be matched: 681.


#### 2) Check for 90% match


 Split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [94]:
#Cycle for words 90% 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif in film_combine_diff:
    
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    #loop through titles from animelist that are different 
    for title in title_list_combine:
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]     

In [95]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 68.
The number of film that need to be matched: 626.


#### 3) Check for 90% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [96]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.9, then add title to list 
            if counter/ length_word > 0.90:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]        

In [97]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 71.
The number of film that need to be matched: 623.


#### 4) Check for 75% match with numeric and alphabetic character


 Remove non numeric and non alphabetic, split titles into lists, look for 75% match based on revenue title and match those with only 1 potential match

In [98]:
#Cycle for words 75% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
        #Remove special character and add space
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.75, then add title to list 
            if counter/ length_word > 0.75:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]       

In [99]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 86.
The number of film that need to be matched: 606.


#### 5) Check for 90% match alphabetic character


 Remove non alphabetic, split titles into lists, look for 90% match based on revenue title and match those with only 1 potential match

In [100]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 
 
for title_dif in film_combine_diff:

    #Remove special character and numbers and add space
    title_dif1 = re.sub(r'[^A-Za-z ]+', ' ', title_dif)
    split_word = (title_dif1.split())
    length_word = len(split_word)
    list_ = []
    
    for title in combine_show_diff:
        
        #Remove special character and numbers and add space
        title1 = re.sub(r'[^A-Za-z ]+', ' ', title)
        if any(word in title1 for word in split_word):
            counter =0
            for word in split_word:
                if word in title1:
                    counter += 1 
            if counter/ length_word > 0.90:
                list_.append(title)
            else:
                pass
            

            
            
    if len(list_) > 0:
        dictionary[title_dif] = list_
    else:
        pass
        
## Replace the datframe by dictionary

for key in dictionary:
  
    if len(dictionary[key]) == 1:
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]                        
                
        

In [101]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 87.
The number of film that need to be matched: 604.


#### 6) Check for word for word match keeping only alphabet character


 Remove non alphabetic, look for 100% matches, and match those with only 1 potential match

In [102]:
#Cycle for words 90% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1rev_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1rev_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove everything but alphabet character
    title_dif = re.sub(r'[^A-Za-z]+', '', title_dif1)
        
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
         #Remove everything but alphabet character
        title = re.sub(r'[^A-Za-z]+', '', title1)
        
        #Check if there is any match between anime_1rev_df_rev and animelist
        if title == title_dif:
            list_.append(title1)

            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]                      


In [103]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 88.
The number of film that need to be matched: 98.


#### 7) Check for 66% match with numeric and alphabetic character and words counts of potential matches does not exceed word count of film


Remove nonnumeric and non alphabetic, split titles into lists, and look for 66% match based on revenue title while limiting total count of word in animelist title to total count of revenue title. Lastly match those with only 1 potential match

In [104]:
#Cycle for words 66% and keep only numeric and alphabet 

#Create a dictionary to hold title from anime_1film_df_rev as key and titles from combined animelist as those that match X% 


dictionary = {}

#Cycle through title from anime_1film_df_rev that are different 

for title_dif1 in film_combine_diff:
    
    #Remove special character and add space
    title_dif = re.sub(r'[^A-Za-z0-9 ]+', ' ', title_dif1)
    #split words into list 
    split_word = (title_dif.split())
    
    #Get number of words in list to compare
    length_word = len(split_word)
    
    #Create temporary list to hold titles from combines animelist
    list_ = []
    
    
    #loop through titles from animelist that are different 
    for title1 in title_list_combine:
        
        #Remove special character and add space
        title = re.sub(r'[^A-Za-z0-9 ]+', ' ', title1)
        #split words into list 
        split_title = (title.split())
    
        #Get number of words in list to compare
        length_word_title = len(split_title)
        
        #Check if there is any match between anime_1film_df_rev and animelist
        if any(word in title for word in split_word):
            
            # if so, count how many times there is a match for title words in anime_1film_df_rev within animelist
            counter =0
            
            for word in split_word:
                if word in title:
                    counter += 1 
            

            # if count/ length of word > 0.66 and total word count is less than or equal to film title word count, then add title to list 
            if counter/ length_word > 0.66 and length_word_title/length_word<= 1.0:
                list_.append(title1)
            else:
                pass
            
         
    # if length of list is greater than 0, create key and values to dictionary   
    if len(list_) > 0:
        dictionary[title_dif1] = list_
    else:
        pass
        
## Replace the datframe by dictionary if total length of values in key is 1 
 
for key in dictionary:
    if len(dictionary[key]) == 1:
 
        websrapped_df.loc[websrapped_df['film_title'] == key,'film_title'] = dictionary[key][0]         

In [105]:
# movie title comparison output as list
title_list_film,title_list_combine,film_combine_same,film_combine_diff,combine_film_diff = not_show_title_comparison(websrapped_df,'film_title',combined_leftover_df,'title','type')

print(f'The number of film that have the same titles is: {len(film_combine_same)}.')
print(f'The number of film that need to be matched: {len(film_combine_diff)}.')

The number of film that have the same titles is: 94.
The number of film that need to be matched: 91.


### Merge webscrap data to combined dataframe


Dataset will be merged all tables using left join and titles as common key. 

In [106]:
# Create df for concate by using id to avoid duplicates 
condition_1 = anime_3_combine['type']!='TV'
condition_2 = ~anime_3_combine['title'].duplicated(keep=False)

not_show_not_duplicated_anime_df = anime_3_combine[(condition_1 & condition_2)].copy()

# create a dictionary for title to anime id

dictionary_anime_id_shows={}

    #Loop to fill dictionary
for i,row in not_show_not_duplicated_anime_df.iterrows():
    dictionary_anime_id_shows[row['title']] = i


#Create empty dataframe to fill with corrected index and left over to manual observe 
    
concated_film2_df = pd.DataFrame(columns =websrapped_df.columns )
leftover_film2_df = pd.DataFrame(columns =websrapped_df.columns )

# loop through show dataframe to change index to matching index
    #use title for condition to change and add to new dataframe
unique_titles = dictionary.keys()
for i, row in websrapped_df.iterrows():
    unique_titles = dictionary_anime_id_shows.keys()
    if row['film_title'] in unique_titles:
        
        new_index = dictionary_anime_id_shows[row['film_title']]
        concated_film2_df.loc[new_index,:] = row
        
        dictionary_anime_id_shows.pop(row['film_title'])
    else:
        leftover_film2_df.loc[len(leftover_tv_df.index),:] = row

In [107]:
#Merge data using left join on anime_final_combine
anime_final_combine = pd.merge(anime_final_combine,concated_film2_df,how= 'left', left_index=True,right_index=True)

In [108]:
#Check first 3 row and infro
display(anime_final_combine.tail(3))
display(anime_final_combine.info())

,title,title_english,title_synonyms,type,source,episodes,status,duration,rating,related,...,Year,Source Material,Animation Studio,Publisher,Distributor,Movie Title,Type_y,Release Date,film_title,film_revenue
anime_id,,,,,,,,,,,,,,,,,,,,,
51306,c danchi,NaN,NaN,TV,Original,NaN,Not yet aired,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51307,bokura no yoake,NaN,NaN,Movie,Manga,1.0,Not yet aired,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
51333,baise shandian,NaN,NaN,ONA,Original,NaN,Not yet aired,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19132 entries, 1 to 51333
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             19132 non-null  object 
 1   title_english     5724 non-null   object 
 2   title_synonyms    8937 non-null   object 
 3   type              19132 non-null  object 
 4   source            19132 non-null  object 
 5   episodes          18715 non-null  float64
 6   status            17826 non-null  object 
 7   duration          18400 non-null  object 
 8   rating            18400 non-null  object 
 9   related           14478 non-null  object 
 10  producer          8288 non-null   object 
 11  licensor          3373 non-null   object 
 12  studio            11861 non-null  object 
 13  genre             17775 non-null  object 
 14  aired_start       18777 non-null  object 
 15  aired_end         16370 non-null  object 
 16  revenue           2310 non-null   float6

None

After the data are merged, the data from `film_revenue` will be used to update the feature `revenue`. Afterward, the `film_revenue` feature will be dropped. 

In [109]:
# replace missing data with `film_revenue`
anime_final_combine.loc[anime_final_combine['revenue'].isna(),['revenue']] =\
    anime_final_combine.loc[anime_final_combine['revenue'].isna(),['film_revenue']].values

#Drop film_revenue
anime_final_combine.drop(columns = ['film_revenue'], inplace =True)

Turn all zero values in `revenue` to null value

In [110]:
anime_final_combine.loc[anime_final_combine['revenue']==0,'revenue'] =np.nan

### Review the final dataset to condense for EDA purposes

The anime_final_combine dataset will be reviewed for repeated information. Repeated information will be deleted from the dataset.

In [111]:
#Check first 3 row and infro
print('First 3 data rows')
display(anime_final_combine.head(3))
print('\nThe information of the data by columns:')
display(anime_final_combine.info())
print('\nThe percentage of missing values by columns:')
display(anime_final_combine.isna().mean()*100)

First 3 data rows


,title,title_english,title_synonyms,type,source,episodes,status,duration,rating,related,...,Type_x,Year,Source Material,Animation Studio,Publisher,Distributor,Movie Title,Type_y,Release Date,film_title
anime_id,,,,,,,,,,,,,,,,,,,,,
1,cowboy bebop,Cowboy Bebop,NaN,TV,Original,26.0,Finished Airing,24 min. per ep.,R - 17+ (violence & profanity),"{'Adaptation': [{'mal_id': 173, 'type': 'manga...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,cowboy bebop tengoku no tobira,Cowboy Bebop: The Movie,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,Movie,Original,1.0,Finished Airing,1 hr. 54 min.,R - 17+ (violence & profanity),"{'Parent story': [{'mal_id': 1, 'type': 'anime...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,trigun,Trigun,NaN,TV,Manga,26.0,Finished Airing,24 min. per ep.,PG-13 - Teens 13 or older,"{'Adaptation': [{'mal_id': 703, 'type': 'manga...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



The information of the data by columns:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19132 entries, 1 to 51333
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             19132 non-null  object 
 1   title_english     5724 non-null   object 
 2   title_synonyms    8937 non-null   object 
 3   type              19132 non-null  object 
 4   source            19132 non-null  object 
 5   episodes          18715 non-null  float64
 6   status            17826 non-null  object 
 7   duration          18400 non-null  object 
 8   rating            18400 non-null  object 
 9   related           14478 non-null  object 
 10  producer          8288 non-null   object 
 11  licensor          3373 non-null   object 
 12  studio            11861 non-null  object 
 13  genre             17775 non-null  object 
 14  aired_start       18777 non-null  object 
 15  aired_end         16370 non-null  object 
 16 

None


The percentage of missing values by columns:


title                0.000000
title_english       70.081539
title_synonyms      53.287686
type                 0.000000
source               0.000000
episodes             2.179594
status               6.826260
duration             3.826051
rating               3.826051
related             24.325737
producer            56.679908
licensor            82.369852
studio              38.004391
genre                7.092829
aired_start          1.855530
aired_end           14.436546
revenue             87.434664
2_synopsis           3.240644
3_title_english     61.791762
3_synopsis          10.166214
Series Title        89.007945
Type_x              89.007945
Year                89.007945
Source Material     89.007945
Animation Studio    89.007945
Publisher           89.007945
Distributor         89.007945
Movie Title         97.992891
Type_y              97.992891
Release Date        97.992891
film_title          99.508677
dtype: float64

Copies of data are created to save into csv files after features are dropped. 

In [112]:
#Create copy of dataframe for copy
anime_final_combine_rev = anime_final_combine.copy()

In [113]:
#get columns names
anime_final_combine_rev.columns

Index(['title', 'title_english', 'title_synonyms', 'type', 'source',
       'episodes', 'status', 'duration', 'rating', 'related', 'producer',
       'licensor', 'studio', 'genre', 'aired_start', 'aired_end', 'revenue',
       '2_synopsis', '3_title_english', '3_synopsis', 'Series Title', 'Type_x',
       'Year', 'Source Material', 'Animation Studio', 'Publisher',
       'Distributor', 'Movie Title', 'Type_y', 'Release Date', 'film_title'],
      dtype='object')

In [114]:
# column to drop
final_col_drop = [
    'title_english', 'title_synonyms',
    
    
    '3_title_english','Series Title', 'Type_x',
    'Year', 'Source Material',
    'Movie Title','Type_y', 'Release Date', 'film_title'
    ]
    

'''
Index(['title', 'title_english', 'title_synonyms', 'type', 'source',
       'episodes', 'status', 'duration', 'rating', 'related', 'producer',
       'licensor', 'studio', 'genre', 'aired_start', 'aired_end', 'revenue',
       '2_synopsis', '3_title_english', '3_synopsis', 'Series Title', 'Type_x',
       'Year', 'Source Material', 'Animation Studio', 'Publisher',
       'Distributor', 'Movie Title', 'Type_y', 'Release Date', 'film_title'],
      dtype='object')
'''

#drop columns
anime_final_combine_rev.drop(columns = final_col_drop, inplace = True)



In [115]:
#Keep data for analysis based on title 

anime_final_combine_keep = anime_final_combine_rev.loc[(~anime_final_combine_rev['revenue'].isna()),:]

#Leftover
anime_final_combine_leftover = anime_final_combine_rev.loc[(anime_final_combine_rev['revenue'].isna()),:]


In [116]:
anime_final_combine_keep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2404 entries, 7 to 51236
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   title             2404 non-null   object 
 1   type              2404 non-null   object 
 2   source            2404 non-null   object 
 3   episodes          2401 non-null   float64
 4   status            2397 non-null   object 
 5   duration          2391 non-null   object 
 6   rating            2391 non-null   object 
 7   related           2018 non-null   object 
 8   producer          1638 non-null   object 
 9   licensor          1247 non-null   object 
 10  studio            2327 non-null   object 
 11  genre             2397 non-null   object 
 12  aired_start       2402 non-null   object 
 13  aired_end         2386 non-null   object 
 14  revenue           2404 non-null   float64
 15  2_synopsis        2400 non-null   object 
 16  3_synopsis        2355 non-null   object 

In [117]:
anime_final_combine_keep[anime_final_combine_keep['title'].duplicated(keep=False)].sort_values('title')

,title,type,source,episodes,status,duration,rating,related,producer,licensor,studio,genre,aired_start,aired_end,revenue,2_synopsis,3_synopsis,Animation Studio,Publisher,Distributor
anime_id,,,,,,,,,,,,,,,,,,,,


### Create new CSV files

The dataset will be divided into two set depending on whether a row of data has information about revenue. These datasets will be moved into new files called 'revised_data'. The set with revenue data will be used for EDA. 

The new dataset for EDA is is called 'revised_capstone_data.csv' and the left over data is called 'leftover_data.csv'. 

In [118]:
from pathlib import Path  

filepath = Path('revised_data/revised_capstone_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
anime_final_combine_keep.to_csv(filepath)  

In [119]:
filepath = Path('revised_data/leftover_data.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
anime_final_combine_leftover.to_csv(filepath) 

In [121]:
filepath = Path('revised_data/leftover_data_tv_.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
leftover_tv_df.to_csv(filepath) 

In [123]:
filepath = Path('revised_data/leftover_data_film.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
leftover_film_df.to_csv(filepath) 

In [124]:
filepath = Path('revised_data/leftover_data_film2.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
leftover_film2_df.to_csv(filepath) 